In [ ]:
import olca_ipc as ipc
import olca_schema as schema
import pandas as pd
client = ipc.Client(8080)
client

In [ ]:
# Cargar el archivo Excel (reemplaza 'archivo.xlsx' con el nombre de tu archivo)
archivo_excel = "prueba.xlsx"  # Asegúrate de que el archivo esté en el mismo directorio que tu notebook
df = pd.read_excel(archivo_excel)

In [ ]:
# Seleccionar las primeras 6 filas y columnas de la 'A' a la 'F'
df_filtrado = df.iloc[:8, :6]

# Mostrar los datos filtrados
print(df_filtrado)


In [ ]:
for index, row in df_filtrado.iterrows():
    # Obtener la propiedad de flujo desde la base de datos
    flow_property_ref = client.get(schema.FlowProperty, name=row["Property ref:name"])

    # Crear el FlowPropertyFactor
    flow_property_factor = schema.FlowPropertyFactor(
        flow_property=flow_property_ref,
        conversion_factor=1.0,
        is_ref_flow_property=True
    )

    # Crear el flujo
    material_flow = schema.Flow()
    material_flow.name = row["Nombre"]
    material_flow.description = row["Description"]
    material_flow.flow_type = schema.FlowType[row["Flow type"]]
    material_flow.flow_properties = [flow_property_factor]  # ✅ Asegurar que se asigna correctamente
    material_flow.category = row["Category"]

    # Guardar el flujo en la base de datos
    client.put(material_flow)

    print(f"✅ Flujo '{material_flow.name}'")

In [ ]:
df_p = pd.read_excel(archivo_excel)
df_p_filtrado = df_p.iloc[:8, 15:21]

# Mostrar los datos filtrados
print(df_p_filtrado)

In [ ]:
for index, row in df_p_filtrado.iterrows():
    if pd.isna(row["name"]):
        continue  # Saltar filas sin nombre de proceso

    # Obtener la unidad de masa (gramos) del grupo de unidades de masa
    unit_group = client.get(schema.UnitGroup, name="Units of mass")
    gram_unit = next((u for u in unit_group.units if u.name == row["Unit.1"]), None)

    # Crear el proceso
    _process = schema.Process()
    _process.name = row["name"]
    _process.description = "Proceso generado automáticamente"
    _process.process_type = schema.ProcessType.UNIT_PROCESS
    _process.category = row["category"]
    _process.exchanges = []

    # Crear el intercambio (flujo)
    exchange = schema.Exchange()
    exchange.is_input = row["in/out.1"]  # Determina si es input o output
    exchange.flow = client.get(schema.Flow, name=row["flow"])  # Obtener el flujo
    exchange.flow_property = schema.Ref(name=row["flow"])  # Asignar FlowProperty
    exchange.amount = row["Amount.1"]  # Definir la cantidad desde el DataFrame
    exchange.is_quantitative_reference = not row["in/out.1"]  # Output como referencia
    exchange.unit = gram_unit  # Asignar la unidad recuperada

    # Agregar el flujo al proceso
    _process.exchanges.append(exchange)

    # Guardar el proceso en la base de datos
    client.put(_process)

    print(f"✅ Proceso '{_process.name}' creado exitosamente")

In [ ]:
df_p = pd.read_excel(archivo_excel)
df_p_filtrado = df_p.iloc[:20, 8:14]

# Mostrar los datos filtrados
print(df_p_filtrado)

In [ ]:
for index, row in df_p_filtrado.iterrows():
    if pd.isna(row["Proceso"]):
        continue  # Saltar filas sin nombre de proceso

    # Obtener el proceso existente
    _process = client.get(schema.Process, name=row["Proceso"])
    if not _process:
        print(f"⚠️ Proceso '{row['Proceso']}' no encontrado")
        continue

    # Obtener la unidad correspondiente
    unit_group = client.get(schema.UnitGroup, name="Units of mass")
    gram_unit = next((u for u in unit_group.units if u.name == row["Unit"]), None)

    # Crear el intercambio (flujo de entrada)
    exchange = schema.Exchange()
    exchange.is_input = row["in/out"]  # Es un input
    exchange.flow = client.get(schema.Flow, name=row["Flow"])  # Obtener el flujo
    exchange.flow_property = schema.Ref(name=row["Flow"])  # Asignar FlowProperty
    exchange.amount = row["Amount"]  # Definir la cantidad desde el DataFrame
    exchange.is_quantitative_reference = False  # Input no es referencia
    exchange.unit = gram_unit  # Asignar la unidad recuperada
    exchange.default_provider =  client.get(schema.Process, name= row["provedor "] )
    
    # Verificar si el flujo ya está en el proceso para evitar duplicados
    existing_flows = [ex.flow.name for ex in _process.exchanges]
    if row["Flow"] not in existing_flows:
        _process.exchanges.append(exchange)
    else:
        print(f"🔄 El flujo '{row['Flow']}' ya está en el proceso '{_process.name}', se omite duplicado.")

    # Guardar el proceso actualizado en la base de datos
    client.put(_process)

    print(f"✅ Input agregado al proceso '{_process.name}'")

In [ ]:
df_p = pd.read_excel(archivo_excel)
df_p_filtrado = df_p.iloc[:2, 22:23]

# Mostrar los datos filtrados
print(df_p_filtrado)

In [ ]:
for index, row in df_p_filtrado.iterrows():
    process_ref = client.get_descriptor(schema.Process, name=row["Sistema_producto"])
    config = schema.LinkingConfig(prefer_unit_processes=True, provider_linking=schema.ProviderLinking.PREFER_DEFAULTS)
    product_system_ref = client.create_product_system(process_ref, config)
    product_system_ref

    impact_method_ref = client.get_descriptor(schema.ImpactMethod, '2e5cd15d-d539-3141-a950-56d75df9d579')
    setup = schema.CalculationSetup(target=product_system_ref, impact_method=impact_method_ref)
    result = client.calculate(setup)

    state = result.wait_until_ready()
    print(f"result id: {state.id}")

    for impact in result.get_total_impacts():
        print(f"{impact.impact_category.name}: {impact.amount} {impact.impact_category.ref_unit}")